<a href="https://colab.research.google.com/github/hucarlos08/Mathematics/blob/main/Labs/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Land Surface Temperature (LST) Project Helper Functions
====================================================

This module provides helper functions for obtaining and preparing Landsat
satellite data necessary for LST calculation.

Project Structure:
1. Data Acquisition & Preprocessing (provided)
2. Radiance Conversion (student implementation)
3. Brightness Temperature Calculation (student implementation)
4. Emissivity Estimation (student implementation)
5. Final LST Calculation (student implementation)

Key Equations:

$L_λ = M_L * Q_{cal} + A_L$

$T_B = \frac{K_2}{\ln(\frac{K_1}{L_λ} + 1)}$

$LST = \frac{T_B}{1 + (\frac{λ * T_B}{ρ}) * \ln(ε)}$

In [ ]:
import ee
import geemap
import numpy as np
from datetime import datetime, timedelta
from typing import Tuple, Dict, Optional, Union

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# =============================================================================
# SECTION 1: Helper Functions (PROVIDED)
# These functions help you get the required data from Landsat
# =============================================================================

def get_landsat_collection(
    roi: ee.Geometry,
    start_date: str,
    end_date: str,
    cloud_cover: float = 20
) -> ee.ImageCollection:
    """[Previous documentation remains the same]"""
    collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
        .filterBounds(roi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUD_COVER', cloud_cover)))

    return collection

def get_landsat_parameters(image: ee.Image) -> Dict[str, float]:
    """[Previous documentation remains the same]"""
    properties = image.getInfo()['properties']

    thermal_constants = {
        'K1': properties['K1_CONSTANT_BAND_10'],
        'K2': properties['K2_CONSTANT_BAND_10'],
        'M': properties['RADIANCE_MULT_BAND_10'],
        'A': properties['RADIANCE_ADD_BAND_10'],
    }

    return thermal_constants

def get_required_bands(image: ee.Image) -> ee.Image:
    """[Previous documentation remains the same]"""
    bands = image.select([
        'SR_B4',    # Red band (0.64-0.67 µm)
        'SR_B5',    # NIR band (0.85-0.88 µm)
        'ST_B10',   # Thermal band (10.6-11.19 µm)
        'QA_PIXEL'  # Quality assessment band
    ]).rename([
        'red',
        'nir',
        'thermal',
        'qa'
    ])

    return bands

def calculate_ndvi(image: ee.Image) -> ee.Image:
    """[Previous documentation remains the same]"""
    ndvi = image.normalizedDifference(['nir', 'red']).rename('NDVI')
    return ndvi

# =============================================================================
# SECTION 2: Radiance Conversion (STUDENT TASK)
# Implement the conversion from DN to Radiance
# =============================================================================

def convert_to_radiance(thermal_band: np.ndarray, M: float, A: float) -> np.ndarray:
    """
    Convert thermal band Digital Numbers (DN) to Radiance.

    TODO: Implement this function using the equation:
    L_λ = M * DN + A

    Parameters
    ----------
    thermal_band : np.ndarray
        Thermal band values (DN)
    M : float
        Multiplicative scaling factor
    A : float
        Additive scaling factor

    Returns
    -------
    np.ndarray
        Radiance values
    """
    # YOUR CODE HERE
    pass

# =============================================================================
# SECTION 3: Brightness Temperature Calculation (STUDENT TASK)
# Implement the conversion from Radiance to Brightness Temperature
# =============================================================================

def calculate_brightness_temperature(
    radiance: np.ndarray,
    K1: float,
    K2: float
) -> np.ndarray:
    """
    Convert Radiance to Brightness Temperature.

    TODO: Implement this function using the equation:
    T_B = K2 / ln(K1/L_λ + 1)

    Parameters
    ----------
    radiance : np.ndarray
        Spectral radiance values
    K1 : float
        First thermal conversion constant
    K2 : float
        Second thermal conversion constant

    Returns
    -------
    np.ndarray
        Brightness temperature in Kelvin
    """
    # YOUR CODE HERE
    pass

# =============================================================================
# SECTION 4: Emissivity Estimation (STUDENT TASK)
# Implement the estimation of surface emissivity from NDVI
# =============================================================================

def estimate_emissivity(ndvi: np.ndarray) -> np.ndarray:
    """
    Estimate surface emissivity from NDVI values.

    TODO: Implement this function following these rules:
    - For NDVI < 0.2: Use bare soil emissivity
    - For NDVI > 0.5: Use full vegetation emissivity
    - For values in between: Use a mixture model

    Parameters
    ----------
    ndvi : np.ndarray
        NDVI values

    Returns
    -------
    np.ndarray
        Estimated emissivity values
    """
    # YOUR CODE HERE
    pass

# =============================================================================
# SECTION 5: Land Surface Temperature Calculation (STUDENT TASK)
# Implement the final LST calculation
# =============================================================================

def calculate_lst(
    brightness_temp: np.ndarray,
    emissivity: np.ndarray,
    wavelength: float = 10.8
) -> np.ndarray:
    """
    Calculate Land Surface Temperature.

    TODO: Implement this function using the equation:
    LST = TB / (1 + (λ * TB / ρ) * ln(ε))

    Parameters
    ----------
    brightness_temp : np.ndarray
        Brightness temperature values in Kelvin
    emissivity : np.ndarray
        Surface emissivity values
    wavelength : float, optional
        Central wavelength of thermal band in micrometers

    Returns
    -------
    np.ndarray
        Land Surface Temperature in Celsius
    """
    # YOUR CODE HERE
    pass

# =============================================================================
# Example Usage (PROVIDED)
# =============================================================================

if __name__ == "__main__":
    # Sample point (Mexico City)
    point = ee.Geometry.Point([-99.1332, 19.4326])

    # Get sample data
    bands, params = get_sample_data(point)

    # Create map for visualization
    Map = geemap.Map()
    Map.centerObject(point, 12)

    # Visualize NDVI
    vis_params = {
        'min': -1,
        'max': 1,
        'palette': ['blue', 'white', 'green']
    }
    Map.addLayer(bands.select('NDVI'), vis_params, 'NDVI')

    print("\nLST Calibration Parameters:")
    for key, value in params.items():
        print(f"{key}: {value}")

    # YOUR IMPLEMENTATION HERE:
    # 1. Convert thermal band to radiance
    # 2. Convert radiance to brightness temperature
    # 3. Calculate emissivity
    # 4. Calculate final LST
    # 5. Visualize results

    Map